# El problema de trabajar con valores faltantes

Esto lo abarcamos en el anterior curso de Exploración y detección de valores faltantes.

Problemas que nos podemos encontrar:
1. Ignorar valores faltantes: Introduce **sesgos** en el análisis y modelos. De tal manera que *Sobreestimamos o Subestimamos* al valor real, provoca problemas serios al momento de reportar los resultados.
2. Multiples algoritmos fallarán: Los algoritmos que tenemos disponibles en la red, fallarán; es decir aparecerán errores y otros estarán dando resultados no deseados.

Nosotros hablamos anteriormente sobre la imputación, pero nunca la definimos.

## Imputación

Estimar los valores ausentes con base en los valores válidos de otras variables y/o casos de muestra.

A continuación vamos a proceder a empezar a trabajar:

## Configuración de ambiente de trabajo.

Primero creemos un entorno con conda y después ejecutemos el archivo [requirements.txt](requirements.txt) para instalar las dependencias adecuadas.

```sh
#Crear un entorno llamado detección con python 3.9
conda create --name imputacion python=3.9
#Para que sea más rápido lo haré con mamba
mamba create --name imputacion python=3.9
#activar el entorno
conda activate imputacion
#instalar las dependencias desde requirements.txt
mamba install --file requirements.txt
#crear archivo requirements.txt si no lo tengo
mamba env export > requirements.txt
```

Si no podemos instalar con Conda, podemos usar este código:
```bash
pip install pyjanitor matplotlib missingno nhanes pandas scipy seaborn session-info sklearn statsmodels upsetplot
```

or 

```bash
pip install -r requirements.txt
```

## Importar librerías

In [3]:
import janitor
import matplotlib.pyplot as plt
import missingno
import nhanes.load
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import session_info
import sklearn.compose
import sklearn.impute
import sklearn.preprocessing
import statsmodels.api as sm
import statsmodels.datasets
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.graphics.mosaicplot import mosaic

## Importar funciones personalizadas

In [4]:
#Instala antes nbformat
%run pandas-missing-extension-re.ipynb

La función anterior proviene del curso anterior.

# 1. El problema de trabajar con valores faltantes

In [7]:
airquality_df = (
    sm.datasets.get_rdataset("airquality")
    .data
    .clean_names(
        case_type = "snake"
    )
    .add_column("year",1973)
    .assign(
        date = lambda df: pd.to_datetime(df[["year","month","day"]])    
    )
    .sort_values(by='date')
    .set_index('date')
)
airquality_df

/home/dens/miniforge3/envs/imputacion/lib/python3.9/site-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.assign` instead.
  return method(self._obj, *args, **kwargs)


,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


En el código anterior lo que hicimos fue traer un data set de R, y añadimos una columna con formato de fecha.

Ademas el conjunto de datos lo usaremos en clases posteriores.

### ¿Cuál es el problema?
Imaginemos que ya tenemos los datos y queremos ajustarlos a un modelo, entonces procedemos a realizar lo siguiente:

In [8]:
(
    smf.ols(
        formula="temp ~ ozone",
        data=airquality_df
    )
    .fit()
    .summary()
    .tables[0]
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Wed, 16 Oct 2024",Prob (F-statistic):,2.93e-18
Time:,19:52:49,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


De momento ya tenemos las variables dependientes y el modelo que utilizamos, es decir parece que está bien.

Pero recordemos que algunas variables tienen valores faltantes.

¿Qué pasaría si yo quiero considerar otra variable?

*solar_r*

In [9]:
(
    smf.ols(
        formula="temp ~ ozone + solar_r",
        data=airquality_df
    )
    .fit()
    .summary()
    .tables[0]
)

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Wed, 16 Oct 2024",Prob (F-statistic):,1.47e-16
Time:,19:55:06,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


Todo bien al parecer, no nos da ningún error.

**Pero ese es el problema precisamente de los valores faltantes; es decir no nos arrojan errores**

Pero si existe.

Veamos el número de observaciones en nuestro modelo;
 - Modelo: 116 observaciones
 - Data Frame: 153 observaciones

Entonces cuando creamos el primer modelo **tenemos 116 observaciones**. Y si creamos el segundo, vemos que disminuye a **111 observaciones.**

Es decir también perdimos más filas, y a pesar que queramos comparar los modelos, como normalmente lo haríamos con el $R^2$ no podríamos hacerlo, porque ambos modelos están siendo ajustados a partir de diferentes conjuntos de datos y eso hace que se pierda información. Debido a que los algoritmos descartan ciertos valores, así que cuando nosotros tenemos valores faltantes no es recomendable hacer esto.

Esto es un problema porque puede parecer que es adecuado, ya que no nos arroja ningún *Warning* o *Error*- Por ello, siempre hay que estar atento.

Por eso, en este curso nos enfocaremos en la imputación de valores faltantes:

## Reto: Datos de supervivientes

In [14]:
survival_df = sm.datasets.get_rdataset("flchain","survival").data
survival_df

,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter
0,97,F,1997,5.700,4.860,10,1.7,0,85,1,Circulatory
1,92,F,2000,0.870,0.683,1,0.9,0,1281,1,Neoplasms
2,94,F,1997,4.360,3.850,10,1.4,0,69,1,Circulatory
3,92,F,1996,2.420,2.220,9,1.0,0,115,1,Circulatory
4,93,F,1996,1.320,1.690,6,1.1,0,1039,1,Circulatory
...,...,...,...,...,...,...,...,...,...,...,...
7869,52,F,1995,1.210,1.610,6,1.0,0,4997,0,NaN
7870,52,F,1999,0.858,0.581,1,0.8,0,3652,0,NaN
7871,54,F,2002,1.700,1.720,8,NaN,0,2507,0,NaN
7872,53,F,1995,1.710,2.690,9,NaN,0,4982,0,NaN


In [12]:
survival_df.columns

Index(['age', 'sex', 'sample.yr', 'kappa', 'lambda', 'flc.grp', 'creatinine',
       'mgus', 'futime', 'death', 'chapter'],
      dtype='object')

In [17]:
(
    smf.ols(
        formula="death ~ chapter",
        data=survival_df
    )
    .fit()
    .summary()
    .tables[0]
)

/home/dens/miniforge3/envs/imputacion/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dep. Variable:,death,R-squared:,-inf
Model:,OLS,Adj. R-squared:,-inf
Method:,Least Squares,F-statistic:,-143.5
Date:,"Wed, 16 Oct 2024",Prob (F-statistic):,1.00
Time:,21:59:36,Log-Likelihood:,71215.
No. Observations:,2169,AIC:,-1.424e+05
Df Residuals:,2153,BIC:,-1.423e+05
Df Model:,15,,
Covariance Type:,nonrobust,,


In [16]:
(
    smf.ols(
        formula="death ~ age + chapter",
        data=survival_df
    )
    .fit()
    .summary()
    .tables[0]
)

/home/dens/miniforge3/envs/imputacion/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dep. Variable:,death,R-squared:,-inf
Model:,OLS,Adj. R-squared:,-inf
Method:,Least Squares,F-statistic:,-134.5
Date:,"Wed, 16 Oct 2024",Prob (F-statistic):,1.00
Time:,21:58:37,Log-Likelihood:,69860.
No. Observations:,2169,AIC:,-1.397e+05
Df Residuals:,2152,BIC:,-1.396e+05
Df Model:,16,,
Covariance Type:,nonrobust,,
